# Construction et analyse des intervalles de confiance pour le SoH des tesla


In [ ]:
from transform.raw_results.config import *
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from core.pandas_utils import *
import plotly.graph_objects as go
import plotly.express as px
from transform.raw_results.tesla_results import get_results 


Ce notebook permet de :
- Voir si on peut construire des intervalles de confiances 
- La répartition en fonction du nombre de charges

Conclusion :
- Sur 7969 vin avec des SoH on a:
    - 1310 vin avec un IC de +/- 1%
    - 6163 vin avec un IC de +/- 2.5% soit 77%
    - On a uniquement 202 vin qui ont un IC > +/- 5%
    
    - On à certains vin pour lesquels l'IC est quasi nul car il y'a eu peu de charges (moins de 3 charges environ 215 vin) mais seulement 30 avec un IC très faible.  


## data import

In [ ]:
df = get_results().dropna(subset='soh')

In [ ]:
df.vin.nunique()

## IC build

In [ ]:
def ic_computation(df):
    len_ = df.shape[0]
    m = df['soh'].mean()
    med = df['soh'].median()
    std = df['soh'].std()
    if len_ > 0:
        upper_bound = m + 1.96 * (std/np.sqrt(len_))
        lower_bound = m - 1.96 * (std/np.sqrt(len_))
        
        return (round(lower_bound, 4),round( upper_bound, 4)), len_, med
    return (np.nan, np.nan), len_, med

In [ ]:
# filtre pour supprimer les outliers
ic_df = pd.DataFrame(df[(df['soh'] < 1.1) & (df['soh']>.4)].groupby(['vin'], observed=True)[['soh']].apply(ic_computation))

In [ ]:
l1, number_charges, median = zip(*ic_df[0].values)
lower , upper = zip(*list(l1))

In [ ]:
ic_df['upper'] = list(upper)
ic_df['lower'] = list(lower)
ic_df["number_charges"] = list(number_charges)
ic_df["soh_median"] = list(median)

In [ ]:
ic_df['ic_point_diff'] = ic_df['upper'] - ic_df['lower']

## Graphs

In [ ]:
hist_values, bin_edges = pd.cut(ic_df[ic_df['number_charges'] > 3]['ic_point_diff'], bins=[0, .02, .05, .1, .2, .3, .4, .5, 1], right=False, retbins=True)
hist_counts = hist_values.value_counts().sort_index()
fig = go.Figure(data=[go.Bar(
    x=[f"{round(bin_edges[i], 3)} - {round(bin_edges[i+1], 3)}" for i in range(len(bin_edges)-1)],
    y=hist_counts.values,
    marker=dict(color='blue'),
    text=hist_counts
)])
fig.update_layout(
    title="répartitions des tailles d'IC",
    xaxis_title="Intervale",
    yaxis_title="Frecuence"
)
fig.show()

In [ ]:
px.scatter(ic_df, x='number_charges', y='ic_point_diff', hover_data={'lower': True, 
                                                       'upper': True}, title='taille IC vs Nombre de charges')

In [ ]:
charges_df = ic_df.groupby("number_charges", as_index=False).agg(
    mean_ic_point = ('ic_point_diff', 'mean'),
    median_ic_point = ('ic_point_diff', 'median'),
    max_ic_point = ('ic_point_diff', 'max'),
    
).dropna()

In [ ]:
px.scatter(charges_df, x='number_charges', y='mean_ic_point', title='taille IC moyen par nombre de charges')